<a href="https://colab.research.google.com/github/LCP2022/Arduino0V5642/blob/main/TFProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

# Prompt the user to upload a file
uploaded = files.upload()

Saving image_label.npy to image_label.npy
Saving image_value.npy to image_value.npy


In [3]:
import numpy as np

In [8]:
reshaped_imagearr = np.load("image_value.npy")
one_hot_encoded = np.load('image_label.npy')

print(f"image Reshape{reshaped_imagearr.shape}")
print(f"Label Reshape{one_hot_encoded.shape}")

image Reshape(30, 24, 32, 1)
Label Reshape(30, 3)


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_Atrain, X_test, y_Atrain, y_test = train_test_split(reshaped_imagearr,one_hot_encoded, test_size=0.4, random_state=42)
X_Avalid, X_Atest, y_Avalid, y_Atest = train_test_split(X_test,y_test, test_size=0.5, random_state=42)

print(f"Train Set {len(y_Atrain)}:")
print(f"valid Set {len(y_Avalid)}:")
print(f"Test Set {len(y_Atest)}:")


Train Set 18:
valid Set 6:
Test Set 6:


In [12]:
import tensorflow as tf
from tensorflow import keras

In [13]:
# Define the CNN model
def customCnnmodel(InS,OuTS):
    model = keras.Sequential()
    model.add (keras.layers.Conv2D(2,(3,3), activation='relu', input_shape=InS))
    model.add (keras.layers.MaxPooling2D((2,2)))
    model.add (keras.layers.Flatten())
    model.add (keras.layers.Dense(10, activation='relu'))
    model.add (keras. layers.Dense(OuTS, activation='softmax'))  # Assuming 10 classes for classification
    # Compile the model
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
              metrics=['accuracy'])
    model.summary()
    return model

In [14]:
model = customCnnmodel(X_Atrain[0].shape,3)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 22, 30, 2)         20        
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 15, 2)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 330)               0         
                                                                 
 dense (Dense)               (None, 10)                3310      
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 3363 (13.14 KB)
Trainable params: 3363 (13.14 KB)
Non-trainable params: 0 (0.00 Byte)
______________________

In [15]:
history = model.fit(X_Atrain,y_Atrain,epochs=100,batch_size=32,validation_data=[X_Avalid,y_Avalid])

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.1262 - val_accuracy: 0.1667
Epoch 2/100
1/1 [==============================] - 0s 74ms/step - loss: 1.0977 - accuracy: 0.3333 - val_loss: 1.1201 - val_accuracy: 0.1667
Epoch 3/100
1/1 [==============================] - 0s 57ms/step - loss: 1.0967 - accuracy: 0.3333 - val_loss: 1.1154 - val_accuracy: 0.1667
Epoch 4/100
1/1 [==============================] - 0s 53ms/step - loss: 1.0961 - accuracy: 0.3333 - val_loss: 1.1117 - val_accuracy: 0.1667
Epoch 5/100
1/1 [==============================] - 0s 53ms/step - loss: 1.0956 - accuracy: 0.3333 - val_loss: 1.1090 - val_accuracy: 0.1667
Epoch 6/100
1/1 [==============================] - 0s 62ms/step - loss: 1.0951 - accuracy: 0.3333 - val_loss: 1.1071 - val_accuracy: 0.1667
Epoch 7/100
1/1 [==============================] - 0s 71ms/step - loss: 1.0948 - accuracy: 0.3333 - val_loss: 1.1061 - val_accuracy: 0.1667
Epoch 8/100
1/1 [=====

In [16]:
prediction = model.predict(X_Atest)
predicted_classes = np.argmax(prediction, axis=1)
print(f"Target  {y_Atest}")
print(f"Predict {predicted_classes}")
prediction[0]

1/1 [==============================] - 0s 146ms/step
Target  [[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
Predict [0 0 1 1 0 2]


array([0.6487851 , 0.19896224, 0.1522526 ], dtype=float32)

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("object_model_no_quat.tflite", "wb").write(tflite_model)

16176

In [18]:
import os

In [19]:
basic_model_size = os.path.getsize("object_model_no_quat.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 16176 bytes


In [23]:
model.save("object_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
interpreter = tf.lite.Interpreter("object_model_no_quat.tflite")
interpreter.allocate_tensors()

In [21]:
input_details = interpreter.get_input_details()[0]['index']
output_details = interpreter.get_output_details()[0]['index']


In [22]:
x_value_tensor = tf.convert_to_tensor([X_Atest[0]],dtype=np.float32)
interpreter.set_tensor(input_details,x_value_tensor)
interpreter.invoke()
interpreter.get_tensor(output_details)[0]




array([0.64878446, 0.19896278, 0.15225272], dtype=float32)

In [26]:
def hex_to_array(hex_data,var_name):
    c_str =''

    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n'

    c_str += 'const unsigned int ' + var_name + '_len = ' +  str(len(hex_data)) +';\n'

    c_str += 'const unsigned char ' + var_name + '[] = {'
    hex_array=[]

    for i, val in enumerate(hex_data):
        hex_str = format(val,'#04x')

        if (i+1)<len(hex_data):
            hex_str +=','
        if(i+1)%12 ==0:
            hex_str +='\n'
        hex_array.append(hex_str)

    c_str += '\n' + format(''.join(hex_array)) + "\n};\n\n"
    c_str += '#endif  //' + var_name.upper()+ '_H'

    return c_str

In [27]:
with open("object_model.h",'w') as file :
        file.write(hex_to_array(tflite_model,"object_model"))